In [1]:
import pandas as pd
import sqlite3

# Objectives

- Use SQL subqueries to nest queries
- Identify common SQL dialects and tools
- Query data from web databases

![yo dawg sql meme](images/yodawg.jpeg)

# SQL Subqueries

Like you might nest one function within another in Python, you can nest queries in SQL. We can use a **subquery** within another query to succinctly implement queries that have multiple query steps.

In [2]:
conn = sqlite3.connect('data/flights.db')

In [3]:
connection = sqlite3.Connection('data/flights.db')

In [4]:
type(conn), type(connection)

(sqlite3.Connection, sqlite3.Connection)

## Subqueries in `FROM`

You can use a subquery in the `FROM` clause - this is useful, for example, if you want to apply multiple aggregation functions.

Let say we want to get the average of the number of routes departing from all airports. First we'd need to get the total number of routes departing from all airports, then take the average.

In [5]:
pd.read_sql('''

SELECT source AS departing_airport, COUNT() AS num_departures 
FROM routes
GROUP BY source

''', conn)

,departing_airport,num_departures
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


We can use this query as a subquery, and take the average of the new `number_of_departures` column.

In [6]:
pd.read_sql('''

SELECT AVG(num_departures)
FROM (SELECT source AS departing_airport, COUNT() AS num_departures 
        FROM routes
        GROUP BY source)

''', conn)

,AVG(num_departures)
0,19.848343


## Note: Subqueries are Like New Tables!

If you squint, you'll notice that the subquery is taking the place of where we might put a table!

For example, checkout the SQL we wrote in our first subquery example:

```sql
SELECT 
    AVG(number_of_departures)
FROM (
    SELECT 
        source AS depart_airport
        ,COUNT() AS number_of_departures
    FROM
        routes
    GROUP BY
        source
)
```

We could imagine that some new table that returned by the subquery existed (let's call it `airport_departures`) and be placed in place of the subquery:

```sql
SELECT 
    AVG(number_of_departures)
FROM (
    airport_departures -- Replacing subquery with this hypothetical table
) 
```

You can actually use syntax close to this with **Common Table Expressions (CTEs)** found in the [Level Up section](#Level-Up:-Common-Table-Expressions) below.

## Subqueries in `WHERE`

You can use a subquery in the `WHERE` clause - this is useful, for example, if you want to filter a query based on results from another query.

Let's say that we want to get a table with all of the departures and destinations for the flight routes, but I only want to include flights departing from the five countries with the most airports.

To do this, we'd first need to identify the five countries that have the most airports. 

In [10]:
df_countries= pd.read_sql('''

SELECT country, COUNT()
FROM airports
GROUP BY country
ORDER BY COUNT() DESC
LIMIT 5
            
''', conn)

In [11]:
countries = list(df_countries['country'])
countries

['United States', 'Canada', 'Germany', 'Australia', 'Russia']

I could enter these results into a new query of the routes table to get the data I want.

In [15]:
pd.read_sql('''
SELECT *
FROM routes
    LEFT JOIN airports
        ON routes.source_id = airports.id
WHERE airports.country IN ('United States', 'Canada', 'Germany')
ORDER BY source_id 
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,...,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,1072,4N,341,YOW,100,YZF,196,None,0,737,...,Ottawa,Canada,YOW,CYOW,45.3225,-75.669167,374,-5,A,America/Toronto
1,2063,5T,1623,YOW,100,YFB,55,None,0,73M,...,Ottawa,Canada,YOW,CYOW,45.3225,-75.669167,374,-5,A,America/Toronto
2,2504,7F,2354,YOW,100,YFB,55,None,0,73M,...,Ottawa,Canada,YOW,CYOW,45.3225,-75.669167,374,-5,A,America/Toronto
3,6959,AA,24,YOW,100,CLT,3876,Y,0,CRJ,...,Ottawa,Canada,YOW,CYOW,45.3225,-75.669167,374,-5,A,America/Toronto
4,6960,AA,24,YOW,100,PHL,3752,Y,0,CRJ,...,Ottawa,Canada,YOW,CYOW,45.3225,-75.669167,374,-5,A,America/Toronto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17388,63368,WJ,5399,YNA,94,YZV,202,None,0,BE1,...,Natashquan,Canada,YNA,CYNA,50.19,-61.789167,39,-5,A,America/Toronto
17389,63369,WJ,5399,YNA,94,ZGS,\N,None,0,DHT,...,Natashquan,Canada,YNA,CYNA,50.19,-61.789167,39,-5,A,America/Toronto
17390,63370,WJ,5399,YNA,94,ZKG,8217,None,0,DHT,...,Natashquan,Canada,YNA,CYNA,50.19,-61.789167,39,-5,A,America/Toronto
17391,3703,9M,\N,YOJ,99,YEG,49,None,0,BEH,...,High Level,Canada,YOJ,CYOJ,58.621389,-117.164722,1110,-7,A,America/Edmonton


This approach works but has a few limitations:

- We have to manually enter the countries to filter them
- The list of countries won't update with our data, so we'd have to monitor and manually change them in the future
- We have to look at two separate queries to understand what our code is supposed to do
- We have to run two separate queries, which might take longer than one combined query

A better solution uses a subquery to get the list of 5 countries and feed it into our WHERE clause.

In [16]:
pd.read_sql('''

SELECT r.source AS departing_airport, r.dest AS destination, a.country AS departing_country
FROM routes AS r
    LEFT JOIN airports AS a
        ON r.source_id = a.id
WHERE a.country IN (SELECT country
                    FROM airports
                    GROUP BY country
                    ORDER BY COUNT() DESC
                    LIMIT 5)
ORDER BY departing_country

''', conn)

,departing_airport,destination,departing_country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


## Level Up: Common Table Expressions

Common Table Expressions (CTEs) are a more readable way to implement subqueries, using `WITH` and `AS`.

In [17]:
pd.read_sql('''
WITH top_5_countries AS (SELECT country
                    FROM airports
                    GROUP BY country
                    ORDER BY COUNT() DESC
                    LIMIT 5)

SELECT 
    rt.source AS depart_airport
    ,rt.dest AS destination_airport
    ,ap.country AS depart_country
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    ap.country IN top_5_countries
ORDER BY 
    depart_country
''', conn)

,depart_airport,destination_airport,depart_country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


### Exercise

Create a table listing all airlines that serve the three airports with the most outbound routes.

In [18]:
pd.read_sql('''
SELECT *
FROM airports

''', conn).head(2)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


In [19]:
pd.read_sql('''
SELECT *
FROM routes

''', conn).head(2)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2


In [20]:
pd.read_sql('''
SELECT *
FROM airlines

''', conn).head(2)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N


In [23]:
## Your work here
pd.read_sql("""
WITH top_3 AS (
    SELECT ap.id, ap.name
    FROM airports ap
    JOIN routes r
    ON ap.id = r.source_id
    GROUP BY ap.id
    ORDER BY COUNT() DESC
    LIMIT 3
)

SELECT DISTINCT airline, airline_id
FROM routes
WHERE source_id IN (SELECT id FROM top_3)
ORDER BY airline
            
""", conn)

,airline,airline_id
0,3E,10739
1,3M,\N
2,3U,4608
3,5J,1683
4,8L,2942
...,...,...
97,WN,4547
98,WS,5416
99,Y4,5325
100,Y7,13088


In [26]:
pd.read_sql("""
WITH top_3 AS (SELECT ap.id
                FROM airports ap
            JOIN routes r
            ON ap.id = r.source_id
        GROUP BY ap.id
        ORDER BY COUNT() DESC
        LIMIT 3)

SELECT DISTINCT a.name, a.id, r.airline, r.airline_id
FROM routes r
    LEFT JOIN airlines a
            on a.id = r.airline_id
WHERE r.source_id IN top_3
order by a.name
            
""", conn)

,name,id,airline,airline_id
0,None,None,3M,\N
1,Aer Lingus,837,EI,837
2,AeroMéxico,321,AM,321
3,Aeroflot Russian Airlines,130,SU,130
4,Air Algerie,794,AH,794
...,...,...,...,...
97,Virgin Australia,5360,VA,5360
98,Virgin Express,5333,TV,5333
99,Volaris,5325,Y4,5325
100,WestJet,5416,WS,5416


<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```sql
WITH top_3_airports AS (
    SELECT 
        airports.id 
    FROM 
        airports 
        LEFT JOIN routes
            ON routes.source_id = airports.id
    GROUP BY
        airports.id
    ORDER BY 
        COUNT() DESC
    LIMIT 3
) 

SELECT DISTINCT
    rt.airline
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    rt.source_id IN top_3_airports
```
</details>

# SQL Versions

There is no one version of SQL - there are many versions out there! What you're learning about SQL with SQLite will apply to all of them. Just keep in mind when you apply for jobs that you may see any of these listed in any given job posting, and they are all just different versions of what you know.

## SQL Dialects

As with dialects of spoken languages, SQL dialects have many commonalities but some differences in syntax and functionality.  Here are a few of the major players:

- SQLite (we've already seen this!)
- PostgreSQL (free and open-source!)
- Oracle SQL
- MySQL (half open-souce, half Oracle)
- Microsoft SQL Server
- Transact-SQL (extends MS SQL)

## SQLite Pros & Cons

We use SQLite in this course, but it has some limitations.

### Pros

- Easy to set up
- Easy to share database files
- Uses little memory

### Cons

- Limited functionality for managing users and access permissions
- Not "thread safe": two edits at the same time can mess up your data

In [29]:
# Reopen the database connection
conn = sqlite3.connect('data/flights.db')

# Using LIKE in SQL WHERE
pd.read_sql('''
SELECT country, code, city
FROM airports
WHERE country LIKE '%Sta%'
''', conn)

,country,code,city
0,United States,4I7,Greencastle
1,United States,C91,Dowagiac
2,United States,CDI,Cambridge
3,United States,SUE,Sturgeon Bay
4,United States,0P2,Stewartstown
...,...,...,...
1845,United States,None,Valdez
1846,United States,None,Bainbridge Island
1847,United States,CWT,Chatsworth
1848,United States,DHB,Deer Harbor


In [30]:
# Using LIKE in SQL WHERE
pd.read_sql('''
SELECT country, code, city
FROM airports
WHERE country LIKE 'M_%_%'
''', conn)

,country,code,city
0,Mauritius,MRU,Plaisance
1,Mauritius,RRG,Rodriguez Island
2,Mayotte,DZA,Dzaoudzi
3,Madagascar,TNR,Antananarivo
4,Madagascar,ZVA,Miandrivazo
...,...,...,...
374,Martinique,None,Fort de France
375,Martinique,None,Fort-de-France
376,Martinique,None,Saint Pierre
377,Mexico,None,Puerto Vallarta


In [34]:
#pandas .query and .eval 

# Reopen the database connection
conn = sqlite3.connect('data/flights.db')
airports=pd.read_sql('''
SELECT *
FROM airports
''', conn) 

airports.head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [35]:
airports.query('name == "Goroka"')

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby


In [36]:
airports.loc[airports['name'] == 'Goroka']

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby


In [37]:
samp = airports.sample(100)
samp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 5689 to 4450
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      100 non-null    int64 
 1   id         100 non-null    object
 2   name       100 non-null    object
 3   city       100 non-null    object
 4   country    100 non-null    object
 5   code       69 non-null     object
 6   icao       99 non-null     object
 7   latitude   100 non-null    object
 8   longitude  100 non-null    object
 9   altitude   100 non-null    object
 10  offset     100 non-null    object
 11  dst        100 non-null    object
 12  timezone   100 non-null    object
dtypes: int64(1), object(12)
memory usage: 10.9+ KB


In [39]:
samp['latitude'] = samp['latitude'].astype(float)
samp['longitude'] = samp['longitude'].astype(float)

In [41]:
samp.eval('combined = latitude + longitude', inplace=True)
samp.head()


,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone,combined
5689,5689,7114,Venetie Airport,Venetie,United States,VEE,PAVE,67.008611,-146.366389,574,-9,A,America/Anchorage,-79.357778
5404,5404,6925,Ust-Ilimsk,Ust-Ilimsk,Russia,None,\N,58.116667,102.466667,1244,9,N,Asia/Irkutsk,160.583334
3057,3057,3140,Dundigul,Dundigul,India,None,VODG,17.627178,78.403361,2013,5.5,N,Asia/Calcutta,96.030539
2884,2884,2963,Shpakovskoye,Stavropol,Russia,STW,URMT,45.109165,42.112778,1486,4,N,Europe/Moscow,87.221943
4894,4894,5427,Kerema Airport,Kerema,Papua New Guinea,KMA,AYKM,-7.963610,145.771000,10,10,U,Pacific/Port_Moresby,137.807390


In [42]:
df_comb=samp.eval('combined=latitude + longitude')
df_comb

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone,combined
5689,5689,7114,Venetie Airport,Venetie,United States,VEE,PAVE,67.008611,-146.366389,574,-9,A,America/Anchorage,-79.357778
5404,5404,6925,Ust-Ilimsk,Ust-Ilimsk,Russia,None,\N,58.116667,102.466667,1244,9,N,Asia/Irkutsk,160.583334
3057,3057,3140,Dundigul,Dundigul,India,None,VODG,17.627178,78.403361,2013,5.5,N,Asia/Calcutta,96.030539
2884,2884,2963,Shpakovskoye,Stavropol,Russia,STW,URMT,45.109165,42.112778,1486,4,N,Europe/Moscow,87.221943
4894,4894,5427,Kerema Airport,Kerema,Papua New Guinea,KMA,AYKM,-7.963610,145.771000,10,10,U,Pacific/Port_Moresby,137.807390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,2571,2641,Chacalluta,Arica,Chile,ARI,SCAR,-18.348531,-70.338742,167,-4,S,America/Santiago,-88.687273
192,192,194,Downsview,Toronto,Canada,YZD,CYZD,43.742500,-79.465556,652,-5,A,America/Toronto,-35.723056
1986,1986,2027,Oamaru,Oamaru,New Zealand,OAM,NZOU,-44.970000,171.081667,99,12,Z,Pacific/Auckland,126.111667
920,920,936,Ampampamena,Ampampamena,Madagascar,None,FMNZ,-13.484814,48.632739,49,3,U,Indian/Antananarivo,35.147925


In [32]:
conn.close()


## Extra Resources: SQL Versions

[What Is a SQL Dialect, and Which one Should You Learn?](https://learnsql.com/blog/what-sql-dialect-to-learn/)

[SQLite vs MySQL vs PostgreSQL](https://www.digitalocean.com/community/tutorials/sqlite-vs-mysql-vs-postgresql-a-comparison-of-relational-database-management-systems)

[SQL Dialect Reference](https://en.wikibooks.org/wiki/SQL_Dialects_Reference)

# Web Databases: data.world

For the rest of this lesson, we'll be exploring databases in [data.world](https://data.world/), a web database that we can query using SQL in our browser. For reference, you can see the instructions for creating a new project here: [Getting Started Working with Data at data.world](https://docs.data.world/en/64499-94893-Work-with-data--Projects.html)

## Step 1: Create a data.world account

You will need to enter and verify your email address.

## Step 2: Create a project using this [Austin AirBnB](https://data.world/jonloyens/inside-airbnb-austin) dataset

Navigate to [this page](https://data.world/jonloyens/inside-airbnb-austin) and use the button at the top right of the page to create a new project using the data.

![](images/data_world_austin_airbnb_new_proj.png)

## Step 3: Create a SQL query

In your project, use the "+ Add" button to add a SQL query.

![](images/data_world_add_sql_query.png)

## Step 4: Run a simple SQL query

Try entering "SELECT * FROM listings" and clicking the "Run Query" button in the top right.

## Step 5: Practice using SQL to explore the data

Below are some exercises to practice your SQL skills and help explore the data. You will need to explore the table schemas to complete these exercises. 

You may find it helpful to click on the corresponding .csv files to inspect the data, or look at the columns in each table in the right sidebar.

### Exercise 1: Create a table showing the number of listings in each neighborhood

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>
    
```sql
    SELECT neighbourhood, COUNT(*)
    FROM listings
    GROUP BY neighbourhood
```   
</details>

### Exercise 2: Create a table showing the 20 listings with the most reviews

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```sql
    SELECT *
    FROM listings
    ORDER BY number_of_reviews DESC
    LIMIT 20
```
</details>

### Exercise 3: Create a table showing all of the reviews for listings that are "Bed & Breakfast" property types.

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```sql    
    SELECT *
    FROM reviews
    INNER JOIN listings
        ON listings.id = reviews.listing_id
    WHERE listings.property_type = "Bed & Breakfast"
```
</details>

### Exercise 4: Run your own query using a subquery or CTE.

Note that the syntax and functionality for subqueries and CTEs in data.world are more limited than SQLite, so try creating simple ones.